In [66]:
import csv
import exrex # !pip install exrex
import json 
import pandas as pd
import numpy as np
import re 
import spacy

from tqdm import tqdm
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split

# ebablbe auto-completion
%config Completer.use_jedi = False

## Generate Config File

download model.

In [2]:
# !python -m spacy download en_core_web_trf

generate base_config.cfg and base_config_gpu.cfg on https://spacy.io/usage/training#quickstart

In [3]:
# !python -m spacy init fill-config base_config.cfg config.cfg
# !python -m spacy init fill-config base_config_gpu.cfg config_gpu.cfg

## Gather Data

###  Convert Excel to Txt
upload the excel file to http://15.15.166.35:18888/tree/eipi10/jian-xu3/snps-classification/snps/etl/data/case_note_ner/xlsx

~~~shell
!pip install xlrd==1.2.0
~~~

In [4]:
# !ls -l /tf/eipi10/jian-xu3/snps-classification/snps/etl/data/case_note_ner/xlsx/ 

In [5]:
# ! /tf/eipi10/jian-xu3/snps-classification/snps/etl/xlsx2txt.py  case_note_ner '' 1 '\t'

### Load Txt File

In [6]:

def load_data(text_file):
    names = ['Case_ID', 'Work_Order_Number', 'Created_On', 'Service_Account', 'Due_Date', 
             'Action_Booking_status', 'Action_Date', 'Currently_Worked_by', 'Owner', 'Product_Line', 
             'Follow_Up_Reason_Code', 'Response_Time', 'Work_Order_Type', 'Postal_Code', 'Active_Booking', 
             'Follow_Up_Type', 'Business_Segment', 'Follow_Up_Note', 'Reschedule_Note']
    dtypes = [np.str, np.str, np.str, np.str, np.str, 
              np.str, np.str, np.str, np.str, np.str, 
              np.str, np.str, np.str, np.str, np.str,
              np.str, np.str, np.str, np.str]
    dtypes = {name:dtype for name, dtype in zip(names, dtypes)}
    df = pd.read_table(text_file, names=names, dtype=dtypes, quoting=csv.QUOTE_NONE)

    df.index = range(1, len(df)+1) 
    df['id'] = df.index
      
    return df

text_file='/tf/eipi10/jian-xu3/snps-classification/snps/etl/data/case_note_ner/txt/3RS_July_Extraction_with_notes_004.txt'
df_case = load_data(text_file)

In [7]:
print('-'*50)
print(len(df_case))

print('-'*50)
display(df_case.head(5))

print('-'*50)
display(df_case.describe().T)

print('-'*50)
df_case.info()



--------------------------------------------------
7039
--------------------------------------------------


,Case_ID,Work_Order_Number,Created_On,Service_Account,Due_Date,Action_Booking_status,Action_Date,Currently_Worked_by,Owner,Product_Line,Follow_Up_Reason_Code,Response_Time,Work_Order_Type,Postal_Code,Active_Booking,Follow_Up_Type,Business_Segment,Follow_Up_Note,Reschedule_Note,id
1,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,NaN,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,#NORMAL Part escalated,1
2,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,NaN,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,Please cancel backordered part. Customer has c...,2
3,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,NaN,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,According to bryan this laptop shows repair co...,3
4,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,NaN,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,Back order part,4
5,5061278521,WO-013010632-4,2021-06-16 12:17:00,MR. REGINALD JOSEPH,2021-07-15 14:42:00,Reschedule,2021-07-15 13:09:00,NaN,JOCILYN ANDREA SANDI DURAN,Commercial Premium Notebooks,NaN,NCD,Break Fix,20785,USA-PR-USYWWDWN_E_A_EEG,Reschedule,Computing,NaN,NaN,5


--------------------------------------------------


,count,mean,std,min,25%,50%,75%,max
id,7039.0,3520.0,2032.128605,1.0,1760.5,3520.0,5279.5,7039.0


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7039 entries, 1 to 7039
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Case_ID                7039 non-null   object
 1   Work_Order_Number      7039 non-null   object
 2   Created_On             7039 non-null   object
 3   Service_Account        7038 non-null   object
 4   Due_Date               7017 non-null   object
 5   Action_Booking_status  4223 non-null   object
 6   Action_Date            3408 non-null   object
 7   Currently_Worked_by    887 non-null    object
 8   Owner                  7039 non-null   object
 9   Product_Line           6457 non-null   object
 10  Follow_Up_Reason_Code  2864 non-null   object
 11  Response_Time          7039 non-null   object
 12  Work_Order_Type        7039 non-null   object
 13  Postal_Code            7039 non-null   object
 14  Active_Booking       

In [8]:
print(df_case.groupby(['Follow_Up_Type']).size())

Follow_Up_Type
Additional Parts Required     1666
Onsite Stock Replenishment     376
Partner Request Reassign        16
Request Reassign              2100
Reschedule                     434
Revisit                       2447
dtype: int64


In [74]:
df_case['notes'] = [(('' if pd.isna(crn) else crn) if pd.isna(fun) 
                     else (fun if pd.isna(crn) else fun + '.' + crn))
                    for fun, crn in zip(df_case.Follow_Up_Note, df_case.Reschedule_Note)] 
df_case

,Case_ID,Work_Order_Number,Created_On,Service_Account,Due_Date,Action_Booking_status,Action_Date,Currently_Worked_by,Owner,Product_Line,...,Response_Time,Work_Order_Type,Postal_Code,Active_Booking,Follow_Up_Type,Business_Segment,Follow_Up_Note,Reschedule_Note,id,notes
1,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,...,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,#NORMAL Part escalated,1,#NORMAL Part escalated
2,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,...,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,Please cancel backordered part. Customer has c...,2,Please cancel backordered part. Customer has c...
3,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,...,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,According to bryan this laptop shows repair co...,3,According to bryan this laptop shows repair co...
4,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,...,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,Back order part,4,Back order part
5,5061278521,WO-013010632-4,2021-06-16 12:17:00,MR. REGINALD JOSEPH,2021-07-15 14:42:00,Reschedule,2021-07-15 13:09:00,NaN,JOCILYN ANDREA SANDI DURAN,Commercial Premium Notebooks,...,NCD,Break Fix,20785,USA-PR-USYWWDWN_E_A_EEG,Reschedule,Computing,NaN,NaN,5,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7035,5069292833,WO-014560602,2021-07-16 16:23:00,Kaiser Foundation Health Plan Inc.,2021-07-19 15:09:00,Additional Parts Required,2021-07-16 16:00:00,NaN,6 CSPAE166,A4 PageWide SMB & Enterprise managed,...,Not available,Break Fix,91786,Anthony Garcia,Additional Parts Required,pMPS,NaN,Please order the following part. Qty. 1 B5L04...,7035,Please order the following part. Qty. 1 B5L04...
7036,5069293232,WO-014560595,2021-07-16 16:21:00,CDW - St. Lukes Hospital (Government),2021-07-20 16:26:00,Request Reassign,NaN,NaN,1 CSPAE131,A4 Laserjet SMB,...,Not available,Break Fix,55805,Brock Tharp,Request Reassign,pMPS,NaN,Please reassign to lms,7036,Please reassign to lms
7037,5069293974,WO-014560699,2021-07-16 17:06:00,LITHIA NISSAN OF FRESNO,2021-07-19 18:00:00,Request Reassign,2021-07-19 13:00:00,Aaron Stavinsky,Luis Alejandro Vargas,A4 Laserjet Enterprise,...,NCD,Preventative Maintenance,93710,IS-Aaron Stavinsky,Request Reassign,Printing,NaN,dahlia.garcia@hp.com,7037,dahlia.garcia@hp.com
7038,5069294340,WO-014560697,2021-07-16 17:04:00,"Best Buy co., Inc.",2021-07-20 18:00:00,NaN,NaN,NaN,Subir Das,Retail Solutions Core,...,NCD,Break Fix,95678,PR-David Popadiuc,Revisit,Computing,Please order 842275-001 and have it ship to HF...,NaN,7038,Please order 842275-001 and have it ship to HF...


### Data Cleaning

In [75]:
def filter_out_note_na(df):
    df_filter = df.loc[(df.Follow_Up_Note.notna()) | (df.Reschedule_Note.notna())].copy()
    return df_filter

df_train_test = filter_out_note_na(df_case)
print(len(df_train_test))

# df_train_test['notes'] = [('' if pd.isna(fun) else fun)  + ('' if pd.isna(crn) else ' | ' + crn)
#                           for fun, crn in zip(df_train_test.Follow_Up_Note, df_train_test.Reschedule_Note)] 
df_train_test

6110


,Case_ID,Work_Order_Number,Created_On,Service_Account,Due_Date,Action_Booking_status,Action_Date,Currently_Worked_by,Owner,Product_Line,...,Response_Time,Work_Order_Type,Postal_Code,Active_Booking,Follow_Up_Type,Business_Segment,Follow_Up_Note,Reschedule_Note,id,notes
1,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,...,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,#NORMAL Part escalated,1,#NORMAL Part escalated
2,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,...,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,Please cancel backordered part. Customer has c...,2,Please cancel backordered part. Customer has c...
3,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,...,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,According to bryan this laptop shows repair co...,3,According to bryan this laptop shows repair co...
4,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,...,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,Back order part,4,Back order part
6,5064053283,WO-013536670-2,2021-06-30 11:06:00,Ford Motor Company of Canada,2021-07-02 08:30:00,Request Reassign,2021-07-05 14:45:00,NaN,Gabriel Bustos,A3 Laserjet,...,NCD,IMACD,N9A,Rick Harbarenko,Request Reassign,dMPS,NaN,Please send this case to Corey Harbarenko. Cor...,6,Please send this case to Corey Harbarenko. Cor...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7035,5069292833,WO-014560602,2021-07-16 16:23:00,Kaiser Foundation Health Plan Inc.,2021-07-19 15:09:00,Additional Parts Required,2021-07-16 16:00:00,NaN,6 CSPAE166,A4 PageWide SMB & Enterprise managed,...,Not available,Break Fix,91786,Anthony Garcia,Additional Parts Required,pMPS,NaN,Please order the following part. Qty. 1 B5L04...,7035,Please order the following part. Qty. 1 B5L04...
7036,5069293232,WO-014560595,2021-07-16 16:21:00,CDW - St. Lukes Hospital (Government),2021-07-20 16:26:00,Request Reassign,NaN,NaN,1 CSPAE131,A4 Laserjet SMB,...,Not available,Break Fix,55805,Brock Tharp,Request Reassign,pMPS,NaN,Please reassign to lms,7036,Please reassign to lms
7037,5069293974,WO-014560699,2021-07-16 17:06:00,LITHIA NISSAN OF FRESNO,2021-07-19 18:00:00,Request Reassign,2021-07-19 13:00:00,Aaron Stavinsky,Luis Alejandro Vargas,A4 Laserjet Enterprise,...,NCD,Preventative Maintenance,93710,IS-Aaron Stavinsky,Request Reassign,Printing,NaN,dahlia.garcia@hp.com,7037,dahlia.garcia@hp.com
7038,5069294340,WO-014560697,2021-07-16 17:04:00,"Best Buy co., Inc.",2021-07-20 18:00:00,NaN,NaN,NaN,Subir Das,Retail Solutions Core,...,NCD,Break Fix,95678,PR-David Popadiuc,Revisit,Computing,Please order 842275-001 and have it ship to HF...,NaN,7038,Please order 842275-001 and have it ship to HF...


In [10]:
print(df_train_test.groupby(['Follow_Up_Type']).size())

Follow_Up_Type
Additional Parts Required     1579
Onsite Stock Replenishment     375
Partner Request Reassign        13
Request Reassign              1737
Reschedule                     136
Revisit                       2270
dtype: int64


### Add train test flag

In [11]:
def add_train_test_flag(df, random_state=42, test_size=0.25):
    indexes = df.index
    train_indexes, test_indexes = train_test_split(indexes, test_size=test_size, random_state=random_state)    
    print('train: {}'.format(len(train_indexes)))
    print('test: {}'.format(len(test_indexes)))
    df.loc[train_indexes, 'train_test_flag'] = 'train'
    df.loc[test_indexes, 'train_test_flag'] = 'test'
    return df
    
random_state = 42
df_train_test = add_train_test_flag(df_train_test, random_state=random_state)

df_train_test.tail()

train: 4582
test: 1528


,Case_ID,Work_Order_Number,Created_On,Service_Account,Due_Date,Action_Booking_status,Action_Date,Currently_Worked_by,Owner,Product_Line,...,Work_Order_Type,Postal_Code,Active_Booking,Follow_Up_Type,Business_Segment,Follow_Up_Note,Reschedule_Note,id,notes,train_test_flag
7035,5069292833,WO-014560602,2021-07-16 16:23:00,Kaiser Foundation Health Plan Inc.,2021-07-19 15:09:00,Additional Parts Required,2021-07-16 16:00:00,NaN,6 CSPAE166,A4 PageWide SMB & Enterprise managed,...,Break Fix,91786,Anthony Garcia,Additional Parts Required,pMPS,NaN,Please order the following part. Qty. 1 B5L04...,7035,Please order the following part. Qty. 1 B5L04...,test
7036,5069293232,WO-014560595,2021-07-16 16:21:00,CDW - St. Lukes Hospital (Government),2021-07-20 16:26:00,Request Reassign,NaN,NaN,1 CSPAE131,A4 Laserjet SMB,...,Break Fix,55805,Brock Tharp,Request Reassign,pMPS,NaN,Please reassign to lms,7036,Please reassign to lms,train
7037,5069293974,WO-014560699,2021-07-16 17:06:00,LITHIA NISSAN OF FRESNO,2021-07-19 18:00:00,Request Reassign,2021-07-19 13:00:00,Aaron Stavinsky,Luis Alejandro Vargas,A4 Laserjet Enterprise,...,Preventative Maintenance,93710,IS-Aaron Stavinsky,Request Reassign,Printing,NaN,dahlia.garcia@hp.com,7037,dahlia.garcia@hp.com,test
7038,5069294340,WO-014560697,2021-07-16 17:04:00,"Best Buy co., Inc.",2021-07-20 18:00:00,NaN,NaN,NaN,Subir Das,Retail Solutions Core,...,Break Fix,95678,PR-David Popadiuc,Revisit,Computing,Please order 842275-001 and have it ship to HF...,NaN,7038,Please order 842275-001 and have it ship to HF...,test
7039,5069310370,WO-014563427,2021-07-17 10:26:00,Costco Wholesale Corporation,2021-07-19 16:00:00,Request Reassign,NaN,NaN,7 CSPAE137,NaN,...,Break Fix,60440,PR-Stephen Sandgren,Request Reassign,dMPS,NaN,I am overbooked now and this is only a diagnos...,7039,I am overbooked now and this is only a diagnos...,train


### Get Train, Test Texts

In [12]:
def standardize_text1(text):
    text = text.strip().lower()
    # add space between punctuation
    text = re.sub(r'([.\\!?,\'()\[\]"|;])', r' \1 ', text)
#     # remove characters except for English letters and some punctuations
#     text = re.sub(r"[^A-Za-z\.\-\?\!\,\#\@\% ]", "", text)
    # remove extra spaces
    text = re.sub(r'[" "]+', " ", text)
    return text

In [13]:
texts = [standardize_text1(note) for note in df_train_test.notes]
train_texts = [standardize_text1(note) for note in df_train_test.loc[(df_train_test.train_test_flag=='train')].notes]
test_texts = [standardize_text1(note) for note in df_train_test.loc[(df_train_test.train_test_flag=='test')].notes]

print(len(texts))
print(len(train_texts))
print(len(test_texts))

6110
4582
1528


### Generate NER Label

In [14]:
part_num_patterns = [
 {'expression': r'[a-zA-Z]\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d\d-[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d\d-[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]-\d\d\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d-\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z][a-zA-Z]\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d[a-zA-Z]\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d\d-[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d-\d\d\d-\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]-\d\d\d\d-\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d-[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d[a-zA-Z]-\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z][a-zA-Z]\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d[a-zA-Z]-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d-\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]\d\d[a-zA-Z]-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d-\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z][a-zA-Z]\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d-\d\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d-\d\d-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z]\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z][a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z][a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-[a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z]\d-\d\d\d\d-\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d-\d\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d-\d\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d[a-zA-Z][a-zA-Z]-\d\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d-\d\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d-\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]\d[a-zA-Z]-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d[a-zA-Z]\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d[a-zA-Z]\d\d[a-zA-Z]\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d-\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d[a-zA-Z]\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d\d\d-[a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'},  
]

# remove duplidate data
part_num_patterns1 = {}
for i, pattern in enumerate(part_num_patterns):
    expression = pattern['expression']
    part_num_patterns1[expression] = pattern
    
part_num_patterns = [value for _, value in part_num_patterns1.items()]
print(len(part_num_patterns))

138


In [15]:
common_words_patterns = [
 {'expression': r'order', 'label': 'ACTION'}, 
 {'expression': r'recommended', 'label': 'ACTION'}, 
 {'expression': r'hfpu', 'label': 'ACTION'}, 
 {'expression': r'ship', 'label': 'ACTION'}, 
 {'expression': r'fedex', 'label': 'ACTION'}, 
 {'expression': r'hal', 'label': 'ACTION'}, 
 {'expression': r'site', 'label': 'ACTION'}, 
 {'expression': r'send', 'label': 'ACTION'}, 
 {'expression': r'location', 'label': 'ACTION'}, 
 {'expression': r'pickup', 'label': 'ACTION'}, 
 {'expression': r'sent', 'label': 'ACTION'}, 
 {'expression': r'requested', 'label': 'ACTION'}, 
 {'expression': r'reorder', 'label': 'ACTION'}, 
 {'expression': r'street', 'label': 'ACTION'}, 
 {'expression': r'hfp', 'label': 'ACTION'}, 
 {'expression': r'shipped', 'label': 'ACTION'}, 
 {'expression': r'delivery', 'label': 'ACTION'}, 
 {'expression': r'recommend', 'label': 'ACTION'}, 
 {'expression': r'warehouse', 'label': 'ACTION'}   
]

print(len(common_words_patterns))

19


In [16]:
def generate_entity(texts, patterns={}):   
    
    def entity_from_patterns(text, patterns, pattern_matchs):        
        entities = {}
#         print('-'*50)  
        for pattern in patterns:
            entities = entity_from_pattern(text, pattern, entities, pattern_matchs)
        entities = [value for _, value in sorted(entities.items(), key=lambda item: item[0][0])]
        return entities

    def entity_from_pattern(text, pattern, entities, pattern_matchs):
        expression = pattern['expression']
        label = pattern['label']
        for match in re.finditer(expression, text):
            start, end = match.span()    
            add_key = True
            if start-1 >= 0 and text[start-1]!=' ':
                add_key = False
            if end < len(text) and text[end]!=' ':
                add_key = False                
                
#             drop_keys = []
#             for key, value in entities.items():
#                 start_, end_ = key
#                 if start<start_ and end>end_:
#                     drop_keys.append(key)
#                 if start>start_ and end<end_:
#                     add_key = False
#             for key in drop_keys:
#                 entities.pop(key)

            if add_key: 
#                 print(text, expression)
                pattern_matchs[expression] = pattern_matchs[expression] + 1
                entities[(start, end)] = (start, end, label)
        return entities
        
    examples = []
    pattern_matchs = {pattern['expression']:0 for pattern in patterns}
    for text in texts:
        entities = entity_from_patterns(text, patterns, pattern_matchs)        
        example = (text, {"entities": entities})
        examples.append(example)
    pattern_matchs = [(key, value) for key, value in sorted(pattern_matchs.items(), key=lambda item: -item[1])]
#     pattern_matchs = [(key, value) for key, value in pattern_matchs if value>0]
        
    return examples, pattern_matchs

def show_entity(text_entities, pattern_matchs):
    for example in text_entities:
        print('-'*50)
        text, annotations = example
        print(text) 
        print(*[(text[start:end], start, end, label) for start, end, label in annotations['entities']], sep='\n')
    print('='*50)
    print(*pattern_matchs, sep='\n')

In [17]:
text_entities, pattern_matchs = generate_entity(texts, patterns=part_num_patterns)
train_text_entities, train_pattern_matchs = generate_entity(train_texts, patterns=part_num_patterns)
test_text_entities, test_pattern_matchs = generate_entity(test_texts, patterns=part_num_patterns)

print(len(text_entities))
print(len(train_text_entities)) 
print(len(test_text_entities))

6110
4582
1528


In [20]:
print(len(pattern_matchs))
print(np.sum([len(annotations['entities']) for text, annotations in text_entities]))
print(np.sum([value for key, value in pattern_matchs]))

print('-'*50) 
print(*pattern_matchs, sep='\n')

138
3956
3956
--------------------------------------------------
('[a-zA-Z]\\d[a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 1089)
('[a-zA-Z][a-zA-Z]\\d-\\d\\d\\d\\d-\\d\\d\\d[a-zA-Z][a-zA-Z]', 643)
('[a-zA-Z]\\d\\d\\d\\d\\d-\\d\\d\\d', 547)
('[a-zA-Z][a-zA-Z]\\d\\d\\d-\\d\\d\\d\\d\\d', 483)
('\\d\\d\\d\\d\\d\\d-\\d\\d\\d', 400)
('[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d[a-zA-Z]', 271)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d\\d\\d\\d', 199)
('[a-zA-Z]\\d\\d\\d\\d-\\d\\d\\d\\d\\d', 122)
('\\d\\d\\d\\d-\\d\\d\\d\\d', 83)
('\\d[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 39)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d[a-zA-Z]\\d\\d', 23)
('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\\d\\d\\d[a-zA-Z]\\d\\d[a-zA-Z]', 21)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d[a-zA-Z][a-zA-Z]\\d\\d\\d\\d\\d', 7)
('\\d\\d-\\d\\d\\d\\d\\d-\\d\\d\\d', 5)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d[a-zA-Z][a-zA-Z]\\d', 5)
('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d[a-zA-Z]', 5)
('[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z][a-

### Data Augmentation

In [22]:
expression = '[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d-\\d\\d\\d\\d'
exrex.getone(expression).lower()

'wse-709-0229'

In [51]:
def get_augument_rates(patterns, pattern_matchs):
    pattern_matchs = { expression:count  for expression, count in pattern_matchs }
    pattern_counts = [pattern_matchs[pattern['expression']] for pattern in patterns] 
    pattern_counts = np.array([300+np.sqrt(count-300) if count>300 else count for count in pattern_counts ] )
    pattern_counts = max(pattern_counts) - pattern_counts
    augument_rates = pattern_counts/np.sum(pattern_counts)
    
    text = ['{:97s}{:4d}{:11.7f}'.format(pattern['expression'],  pattern_matchs[pattern['expression']], rate) 
            for pattern, rate in zip(patterns, augument_rates)]
    print(*text, sep='\n') 
    
    return augument_rates

part_num_augument_rates = get_augument_rates(part_num_patterns, pattern_matchs) 

[a-zA-Z]\d\d\d\d\d-\d\d\d                                                                         547  0.0002884
\d\d\d\d\d\d-\d\d\d                                                                               400  0.0004217
[a-zA-Z]\d\d\d\d\d-[a-zA-Z]\d\d                                                                     0  0.0076484
\d\d\d\d\d\d-[a-zA-Z][a-zA-Z]\d                                                                     0  0.0076484
\d\d\d\d\d\d-\d[a-zA-Z]\d                                                                           0  0.0076484
\d\d\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z]                                                               0  0.0076484
[a-zA-Z]\d\d\d\d\d-[a-zA-Z][a-zA-Z]\d                                                               3  0.0075785
\d\d\d\d\d\d-[a-zA-Z]\d\d                                                                           0  0.0076484
[a-zA-Z][a-zA-Z]-\d\d\d\d\d-\d\d                                                                

In [54]:
def augment(data_entities, patterns, augument_rates, augument_num=40000):
    i = 0
    new_texts = []
    
    augment_matches = {}
    while i<augument_num:
        data_index = int(np.random.randint(len(data_entities), size=1))
        text, annotations = data_entities[data_index]
        entities = [(start, end, label) for start, end, label in annotations['entities'] if label=='PART_NUM']        
        if len(entities)<=0:
            continue
        entities.reverse()
        new_text = text
        for start, end, label in entities:                       
            pattern_index = int(np.random.choice(a=len(augument_rates), size=1, replace=True, p=augument_rates))
            pattern = patterns[pattern_index]['expression']
            part_num = exrex.getone(pattern).lower()
            new_text = new_text[0:start] + part_num + new_text[end:] 
            augment_matches[pattern] = augment_matches.get(pattern, 0) + 1
        new_texts.append(new_text)   

        i = i + 1
        if i % 5000==0:
            print(f'{i}/{augument_num}')
    augment_matches = [(key, value) for key, value in sorted(augment_matches.items(), key=lambda item: -item[1])]
    return new_texts, augment_matches
        
    
sample_augment_texts, sample_augment_matches =  augment(train_text_entities[0:10], part_num_patterns,       
                                                        part_num_augument_rates, augument_num=10)

In [56]:
train_augment_texts, train_augment_matches = augment(train_text_entities, part_num_patterns, 
                                                     part_num_augument_rates, augument_num=9000)
train_augment_text_entities, train_augment_pattern_matchs = generate_entity(train_augment_texts, 
                                                                            patterns=part_num_patterns)

print(len(train_augment_texts))
print(np.sum([value for key, value in train_augment_pattern_matchs]))
print(np.sum([len(annotations['entities']) for text, annotations in train_augment_text_entities]))

5000/9000
9000
13261
13261


In [59]:
test_augment_texts, test_augment_matches = augment(test_text_entities, part_num_patterns, 
                                                   part_num_augument_rates, augument_num=3000)
test_augment_text_entities, test_augment_pattern_matchs = generate_entity(test_augment_texts, 
                                                                          patterns=part_num_patterns)

print(len(test_augment_texts))
print(np.sum([value for key, value in test_augment_pattern_matchs]))
print(np.sum([len(annotations['entities']) for text, annotations in test_augment_text_entities]))

3000
4475
4475


In [60]:
train_data = train_text_entities
train_data_aug = train_text_entities + train_augment_text_entities
test_data = test_text_entities
test_data_aug = test_text_entities + test_augment_text_entities

print(len(train_data))
print(len(train_data_aug))
print(len(test_data))
print(len(test_data_aug))

4582
13582
1528
4528


## Ouput Data

### Save Data 

In [76]:
df_case.to_pickle('./data/df_case.pkl') 
df_train_test.to_pickle('./data/df_train_test.pkl') 

In [72]:
with open('./data/part_num_patterns.json', 'w') as outfile:
    json.dump(part_num_patterns, outfile, indent=2)
    
with open('./data/common_words_patterns.json', 'w') as outfile:
    json.dump(common_words_patterns, outfile, indent=2)
    
with open('./data/texts.json', 'w') as outfile:
    json.dump(texts, outfile, indent=2)    
    
with open('./data/train_texts.json', 'w') as outfile:
    json.dump(train_texts, outfile, indent=2)
    
with open('./data/test_texts.json', 'w') as outfile:
    json.dump(test_texts, outfile, indent=2)    
    
with open('./data/train_text_entities.json', 'w') as outfile:
    json.dump(train_text_entities, outfile, indent=2)
    
with open('./data/test_text_entities.json', 'w') as outfile:
    json.dump(test_text_entities, outfile, indent=2)    
    


### Generate NER data

In [63]:
def generate_data(data):
    nlp = spacy.blank("en") # load a new spacy model
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(data): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)
    return db
    
train_spacy = generate_data(train_data)
train_spacy.to_disk("./data/train.spacy") # save the docbin object
test_spacy = generate_data(test_data) 
test_spacy.to_disk("./data/test.spacy") # save the docbin object
train_spacy = generate_data(train_data_aug)
train_spacy.to_disk("./data/train_aug.spacy") # save the docbin object
test_spacy = generate_data(test_data_aug) 
test_spacy.to_disk("./data/test_aug.spacy") # save the docbin object

100%|██████████| 4528/4528 [00:03<00:00, 1180.80it/s]
